In [13]:
import pandas as pd
from cleantext import clean
from sklearn.model_selection import train_test_split
import datasets

In [9]:
model_name = 't5_sep'
sep = '</s>'

if model_name in ['t5_sep', 't5']:
    summarize = 'summarize: '
else:
    summarize = ''

In [2]:
def clean_phrase(phrase):
    return clean(phrase,
                 lower            = False,
                 no_line_breaks   = True,
                 no_emoji         = True,
                 no_urls          = True,
                 no_phone_numbers = False,
                 lang             = 'en')

In [10]:
rt = pd.read_json("./../rottentomatoes.json")

critics_list = rt._critics.tolist()
critics_cons = rt._critic_consensus.tolist()

critics_concatened = []
critics_summarize = []
for i, critics in enumerate(critics_list):
    tmp_string = ''
    for critic in critics.values():
        tmp_string = tmp_string + clean_phrase(critic) + sep
    critics_concatened.append(summarize + tmp_string[:-len(sep)])
    critics_summarize.append(critics_cons[i])
    
rt['input'] = critics_concatened
rt['summary'] = critics_summarize

new_rt = rt.drop(columns=['_movie_name', '_critics', '_critic_consensus', '_movie_id'])

new_rt.head()

,input,summary
0,summarize: A riveting reconstruction of the st...,"A powerful, documentary-like examination of th..."
1,summarize: To borrow a putdown from a past vic...,Poor plot development and slow pacing keep 54 ...
2,"summarize: Hugely entertaining, remake that's ...","While it hews closely to the 1984 original, Cr..."
3,"summarize: This frank, funny, tender film both...","Tender, funny, and touching, The Sessions prov..."
4,summarize: Chronically impassive and faultless...,Patrice Chéreau's exquisite rendering of Josep...


In [14]:
train, test = train_test_split(new_rt, test_size=0.2, random_state=72)
dataset = \
datasets.DatasetDict({
    'train'     : datasets.Dataset.from_pandas(train, preserve_index=False),
    'test'      : datasets.Dataset.from_pandas(test, preserve_index=False)
    })
dataset.save_to_disk(f'./{model_name}_rt')

Saving the dataset (0/1 shards):   0%|          | 0/2984 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/747 [00:00<?, ? examples/s]